In [1]:
import pandas as pd
import os

In [3]:
# Загрузка данных
df = pd.read_excel(r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\raw\01.07.2025.xlsx", sheet_name="КО")

In [5]:
df

,№ п/п,Регистрационный номер,Тип кредитной организации,Наименование организации,Дата регистрации Банком России,ИНН,ОГРН,Адрес организации,Вид деятельности
0,1,1,NaN,"Акционерное общество ""ЮниКредит Банк""",1989-10-20,7710030411,1027739082106,"119034, г. Москва, Пречистенская набережная, д.9",Банковская деятельность
1,2,2,NaN,"Акционерное Общество ""Коммерческий Акционерный...",1988-08-26,7831000098,1027800000250,"191002 Санкт-Петербург, Владимирский проспект,...",Банковская деятельность
2,3,21,NaN,"Общество с ограниченной ответственностью ""Прим...",1988-11-10,2540015598,1022500001325,"690091, Приморский край, г. Владивосток, ул.Се...",Банковская деятельность
3,4,53,NaN,"Публичное Акционерное Общество Банк ""АЛЕКСАНДР...",1989-01-20,7831000080,1027800000194,"191119, Санкт-Петербург, Загородный пр., 46, л...",Банковская деятельность
4,5,67,NaN,"Акционерный коммерческий банк ""Энергобанк"" (ак...",1989-03-02,1653011835,1021600000289,"420111, Республика Татарстан, г.Казань, ул.Пу...",Банковская деятельность
...,...,...,...,...,...,...,...,...,...
347,348,3553,расчетная НКО,Общество с ограниченной ответственностью Расче...,2025-03-21,9728153016,1257700136100,"117292, г. Москва, ул. Ивана Бабушкина, д. 16А",Банковская деятельность
348,349,3554,расчетная НКО,"Общество с ограниченной ответственностью ""Расч...",2025-03-26,9715508328,1257700145614,"127018, город Москва, улица Складочная, дом 1,...",Банковская деятельность
349,350,3555,расчетная НКО,Общество с ограниченной ответственностью Расче...,2025-04-15,9702075356,1257700178790,"Российская Федерация, г. Москва, Последний пер...",Банковская деятельность
350,351,3556,расчетная НКО,АКЦИОНЕРНОЕ ОБЩЕСТВО РАСЧЕТНАЯ НЕБАНКОВСКАЯ КР...,2025-04-15,9704259704,1257700178768,"125009, г. Москва, вн.тер.г. муниципальный окр...",Банковская деятельность


In [ ]:
from dadata import Dadata


In [ ]:
TOKEN = "37410c9cad46d9c0a5a0e81cd2c264910c023afd"
SECRET = "a6a159b67c8a37205d9588ffb8a2778c9bd2bb90"
dadata = Dadata(TOKEN, SECRET)

In [ ]:
def get_address_by_inn_ogrn(inn=None, ogrn=None):
    """Поиск адреса организации по ИНН или ОГРН через DaData."""
    if not (inn or ogrn):
        return None
    
    try:
        query = str(inn) if inn else str(ogrn)
        result = dadata.suggest("party", query, count=1)
        if result and result[0].get('data'):
            return result[0]['data'].get('address', {}).get('value')
    except Exception as e:
        print(f"Ошибка при поиске по ИНН/ОГРН {inn}/{ogrn}: {e}")
    return None

def clean_address(address_str, inn=None, ogrn=None):
    """Чистка адреса с приоритетом по ИНН/ОГРН."""
    # Сначала пробуем найти адрес по ИНН/ОГРН
    legal_address = get_address_by_inn_ogrn(inn, ogrn)
    
    # Если нашли через ИНН/ОГРН, используем его
    if legal_address:
        cleaned = dadata.clean("address", legal_address)
    else:
        # Иначе чистим исходный адрес
        cleaned = dadata.clean("address", str(address_str))
    
    return {
        'source': address_str,
        'legal_address_by_inn_ogrn': bool(legal_address),
        'result': cleaned.get('result'),
        'postal_code': cleaned.get('postal_code'),
        'federal_district': cleaned.get('federal_district'),
        'region': cleaned.get('region_with_type'),
        'area': cleaned.get('area_with_type'),
        'city': cleaned.get('city_with_type'),
        'street': cleaned.get('street_with_type'),
        'house': cleaned.get('house'),
        'geo_lat': cleaned.get('geo_lat'),
        'geo_lon': cleaned.get('geo_lon')
    }

# Обработка адресов
results = []
for idx, row in df.iterrows():
    address = row['Адрес организации']
    inn = row.get('ИНН')
    ogrn = row.get('ОГРН')
    
    result = clean_address(address, inn, ogrn)
    results.append(result)
    
    if (idx + 1) % 10 == 0:
        print(f"Обработано {idx + 1}/{len(df)}")

In [ ]:
# Добавляем результаты в DataFrame
result_df = pd.DataFrame(results)
final_df = pd.concat([df, result_df], axis=1)

# Сохраняем
output_path = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed\processed_addresses.xlsx"
final_df.to_excel(output_path, index=False)
print(f"Готово! Результат сохранён в {output_path}")

In [ ]:
# Сохраняем
output_path = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed\processed_addresses.xlsx"
final_df.to_excel(output_path, index=False)
print(f"Готово! Результат сохранён в {output_path}")